# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AROUOZZ1187B9ABE51,40.79195,-73.94512,"New York, NY [Spanish Harlem]",Willie Bobo,SOBZBAZ12A6D4F8742,Spanish Grease,168.25424,1997


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values
song_data

array([['SOBZBAZ12A6D4F8742', 'Spanish Grease', 'AROUOZZ1187B9ABE51',
        1997, 168.25424]], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
song_table_insert

'\nINSERT INTO songs\n(id, title, artist_id, year, duration)\nVALUES(%s, %s, %s, %s, %s)\n'

In [10]:
cur.execute(song_table_insert, song_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AROUOZZ1187B9ABE51,40.79195,-73.94512,"New York, NY [Spanish Harlem]",Willie Bobo,SOBZBAZ12A6D4F8742,Spanish Grease,168.25424,1997


In [12]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values
artist_data

array([['AROUOZZ1187B9ABE51', 'Willie Bobo',
        'New York, NY [Spanish Harlem]', 40.79195, -73.94512]],
      dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files('data/log_data')

In [15]:
filepath = log_files[0]

In [16]:
df = pd.read_json(filepath, lines=True) 
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Your Rocky Spine,200,1542931645796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Soziedad Alkoholika,Logged In,Kevin,M,1,Arellano,204.74730,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Va Bien,200,1542931860796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,Franz Ferdinand,Logged In,Kevin,M,2,Arellano,172.01587,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Eleanor Put Your Boots On,200,1542932064796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,Modest Mouse,Logged In,Kevin,M,3,Arellano,209.52771,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Float On,200,1542932236796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
4,Adam Lambert,Logged In,Kevin,M,4,Arellano,266.44853,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Aftermath,200,1542932445796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
import datetime
df = df[df.page == 'NextSong']
df.ts = df.ts.apply(lambda x : datetime.datetime.fromtimestamp(x/1000))
df['hour'] = df.ts.dt.hour
df['day'] = df.ts.dt.day
df['week'] = df.ts.dt.isocalendar().week
df['month'] = df.ts.dt.month
df['year'] = df.ts.dt.year
df['weekday'] = df.ts.dt.weekday
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,hour,day,week,month,year,weekday
0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,...,200,2018-11-22 22:07:25.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66,22,22,47,11,2018,3
1,Soziedad Alkoholika,Logged In,Kevin,M,1,Arellano,204.74730,free,"Harrisburg-Carlisle, PA",PUT,...,200,2018-11-22 22:11:00.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66,22,22,47,11,2018,3
2,Franz Ferdinand,Logged In,Kevin,M,2,Arellano,172.01587,free,"Harrisburg-Carlisle, PA",PUT,...,200,2018-11-22 22:14:24.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66,22,22,47,11,2018,3
3,Modest Mouse,Logged In,Kevin,M,3,Arellano,209.52771,free,"Harrisburg-Carlisle, PA",PUT,...,200,2018-11-22 22:17:16.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66,22,22,47,11,2018,3
4,Adam Lambert,Logged In,Kevin,M,4,Arellano,266.44853,free,"Harrisburg-Carlisle, PA",PUT,...,200,2018-11-22 22:20:45.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66,22,22,47,11,2018,3


In [18]:
#time_data = ()
#column_labels = ()

In [19]:
time_df = df[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']]
time_df = time_df.rename(columns={'ts' : 'time_data'})
time_df.head()

,time_data,hour,day,week,month,year,weekday
0,2018-11-22 22:07:25.796,22,22,47,11,2018,3
1,2018-11-22 22:11:00.796,22,22,47,11,2018,3
2,2018-11-22 22:14:24.796,22,22,47,11,2018,3
3,2018-11-22 22:17:16.796,22,22,47,11,2018,3
4,2018-11-22 22:20:45.796,22,22,47,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
    print(list(row))

[Timestamp('2018-11-22 22:07:25.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:11:00.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:14:24.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:17:16.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:20:45.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:25:11.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:29:30.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:33:55.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:38:05.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:42:32.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:45:07.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 22:49:02.796000'), 22, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 23:34:56.796000'), 23, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-22 23:39:24.796000'), 23, 22, 47, 11, 2018, 3]
[Timestamp('2018-11-23 00:35:36.796000'), 0, 23, 47, 11, 2018,

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].copy()
user_df = user_df.drop_duplicates()
user_df['userId'] = user_df['userId'].apply(lambda x : int(x) if str(x).isdigit() else None)
user_df = user_df.dropna()
user_df.head()

,userId,firstName,lastName,gender,level
0,66,Kevin,Arellano,M,free
12,63,Ayla,Johnson,F,free
15,58,Emily,Benson,F,paid
22,26,Ryan,Smith,M,free
24,16,Rylan,George,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
for index, row in df.iterrows():
    print(row.song)

Your Rocky Spine
Va Bien
Eleanor Put Your Boots On
Float On
Aftermath
Dok si pored mene
Not Enough
Everlong
Si TÃÂº No EstÃÂ¡s AquÃÂ­
All Star
Crazy In Love
More Than A Woman
Yellow
Something About You
Glittering Prize
Don't Cry (Original)
Here_ There And Everywhere
Buck's Boogie
Meddle
Tonight
Terminal Preppie
Pop Champagne
What's The Difference
To Build A Home
Tonight In Flames (Album Version)
Numb (Album Version)
Nostalgia Amnesia
Somebody To Love
You're The One
Edge Of The Ocean
Two Brown Eyes
Stretchin' Out (In A Rubber Band) (LP Version)
Eenie_ Meenie_ Miney_ Mo
Can't Stand Me Now
The Penalty
Love Is A Losing Game
Before I Forget (Album Version)
One More Sad Song
Paradise City
Sehr kosmisch
Cold night for alligators
Interlude (Milo)
Clouds (Of Color Bright Album Version)
Burden In My Hand
Devour (Album Version)
Turn Me Up (Sfaction Version / Feat. Sandy)
Tighten Up
Jump Then Fall
Definitive
Me Pregunto
Lips Of An Angel
Intricacy
Ex-Factor
The Way I Feel (Not Our Master)
Upgrad

In [24]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.